In [ ]:
%load_ext autoreload
%autoreload 2

import pathlib

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.1f' % x)
import scipy.stats
import numpy as np

import plotly.express as px
import plotly.graph_objs as go

from data_handlers.bwm import parse_bwms
from data_handlers.pidstat import parse_pidstat

In [ ]:
experiment_path = pathlib.Path(f"/home/sterz/maci-docker-compose/maci_data/binary_files/44")

In [ ]:
name_replacement = {
}

routing_color_map = {
}

In [ ]:
network_df = parse_bwms(experiment_path)

In [ ]:
procmem_df = parse_pidstat(experiment_path)

In [ ]:
merged = procmem_df.merge(network_df, left_index=True, right_index=True).reset_index()

merged = merged.replace("DTN7Go", "DTN7-go")
merged = merged.replace("mtcp", "MTCP")
merged = merged.replace("quicl", "QUICL")

merged = merged.rename(columns={"Payload Size": "Bundle Size"})
merged = merged.rename(columns={"# Payloads": "# Bundles"})

merged["Software/CLA"] = merged['Software'] + "/" + merged['CLA']

merged["Bundle Size"] = merged["Bundle Size"].replace(1000, "1 kB")
merged["Bundle Size"] = merged["Bundle Size"].replace(100000, "100 kB")
merged["Bundle Size"] = merged["Bundle Size"].replace(1000000, "1 MB")

merged = merged[merged["Loss"] != 50]

In [ ]:
fig = px.box(
    merged,
    x="Bundle Size", 
    y="Mbit/s",
    color="Loss",
    facet_col="Software/CLA",
    facet_row="# Bundles",
    labels={
        "Mbit/s": "",
        "Loss": "Loss (%)",
        "# Bundles": "Bundles",
        "Delivered (%)": ""
    },
    category_orders={
        "Loss": [0, 10, 30],
        "Bundle Size": ["1 kB", "100 kB", "1 MB"],
        "Software/CLA": ["DTN7-go/MTCP", "DTN7-go/QUICL"],
        "Number of Bundles": [10, 100]
    },
    facet_row_spacing=0.08
)

fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", ": ")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Software/CLA: ", "")))

fig.add_annotation(x=-0.05, y=0.35,
                   text="Throughput (Mbit/s)",
                   textangle=-90,
                   xref="paper",
                   yref="paper",
                   font=dict(size=14)
                   )

fig.update_layout(
    margin=dict(l=0, r=0, t=20, b=0),
)

fig.write_image("throughput.pdf", width=500, height=250)
fig

In [ ]:
fig = px.box(
    merged,
    x="Bundle Size", 
    y=merged["%CPU"] / merged["# Node"],
    color="Loss",
    facet_col="Software/CLA",
    facet_row="# Bundles",
    labels={
        "y": "",
        "Loss": "Loss (%)",
        "# Bundles": "Bundles",
        "Delivered (%)": ""
    },
    category_orders={
        "Loss": [0, 10, 30],
        "Bundle Size": ["1 kB", "100 kB", "1 MB"],
        "Software/CLA": ["DTN7-go/MTCP", "DTN7-go/QUICL"],
        "Number of Bundles": [10, 100]
    },
    facet_row_spacing=0.08
)

fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", ": ")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Software/CLA: ", "")))

fig.add_annotation(x=-0.05, y=0.35,
                   text="CPU (%)",
                   textangle=-90,
                   xref="paper",
                   yref="paper",
                   font=dict(size=14)
                   )

fig.update_layout(
    margin=dict(l=0, r=0, t=20, b=0),
    yaxis_range=[0,100]
)

fig.write_image("cpu.pdf", width=500, height=250)
fig

In [ ]:
fig = px.box(
    merged,
    x="Bundle Size", 
    y=(merged["RSS"] / merged["# Node"]) / 1000,
    color="Loss",
    facet_col="Software/CLA",
    facet_row="# Bundles",
    labels={
        "y": "",
        "Loss": "Loss (%)",
        "# Bundles": "Bundles",
        "Delivered (%)": ""
    },
    category_orders={
        "Loss": [0, 10, 30],
        "Bundle Size": ["1 kB", "100 kB", "1 MB"],
        "Software/CLA": ["DTN7-go/MTCP", "DTN7-go/QUICL"],
        "Number of Bundles": [10, 100]
    },
    facet_row_spacing=0.08
)

fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", ": ")))
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Software/CLA: ", "")))

fig.add_annotation(x=-0.05, y=0.35,
                   text="Memory Usage (MB)",
                   textangle=-90,
                   xref="paper",
                   yref="paper",
                   font=dict(size=14)
                   )

fig.update_layout(
    margin=dict(l=0, r=0, t=20, b=0),
    yaxis_range=[0, 700]
)

fig.write_image("memory.pdf", width=500, height=250)
fig